In [1]:
import faiss
import json
from sentence_transformers import SentenceTransformer
import numpy as np
import pandas as pd
from tqdm import tqdm
from pathlib import Path
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import accuracy_score

In [2]:
documents_path = Path("../documents/")
documents_relative_paths = [file.relative_to(documents_path) for file in documents_path.glob('*') if file.is_file()]

In [3]:
model = SentenceTransformer('msmarco-distilbert-base-tas-b-fine-tunned-az-10')

In [4]:
request = model.encode("hello")

In [5]:
embeddings_data = pd.DataFrame(columns=[x for x in range(len(request))])
text_data = pd.DataFrame(columns=["article", "text", "document"])

In [6]:
for index, text in enumerate(documents_relative_paths):
    path_to_document = sorted(Path('..').glob(f'**/{documents_relative_paths[index]}'))[0]
    name = str(path_to_document).replace("..\documents", "")[1:].split(".")[0]
    print(path_to_document)
    with open(str(path_to_document), "r", encoding="utf-8") as file:
        file = json.load(file)

    embeddings_file = pd.DataFrame([model.encode(f"{i}\n{file[i]}") for i in tqdm(file)])
    text_data_1 = pd.DataFrame(columns=["article", "text", "document"])

    text_data_1["article"] = [i for i in file]
    text_data_1["text"] = [file[i] for i in file]
    text_data_1["document"] = name

    text_data = pd.concat([text_data, text_data_1])
    embeddings_data = pd.concat([embeddings_data, embeddings_file])

..\documents\ailə məcəlləsi.json


100%|████████████████████████████████████████████████████████████████████████████████| 194/194 [01:21<00:00,  2.37it/s]


..\documents\borcların icra qeydləri əsasında mübahisəsiz qaydada alınmasına əsas olan sənədlərin siyahısının təsdiq edilməsi haqqında.json


100%|██████████████████████████████████████████████████████████████████████████████████| 42/42 [00:17<00:00,  2.42it/s]


..\documents\dövlət rüsumu haqqında ar qanunu.json


...

..\documents\notariat fəaliyyətinin yoxlanılması qaydalari.json


100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:02<00:00,  2.08it/s]


..\documents\notariat haqqında ar qanunu.json


100%|████████████████████████████████████████████████████████████████████████████████| 101/101 [00:37<00:00,  2.71it/s]


IndexError: list index out of range

In [ ]:
text_data_1

In [11]:
text_data_1 = text_data.reset_index(drop=True).to_csv("E:\Software\Data Science and AI\NLP\Edliyye\Legal Acts Question Answering\NLP project Notarius\sorter\search\other\documents\text_data", index=False)
embeddings_data = embeddings_data.reset_index(drop=True).to_csv("E:\Software\Data Science and AI\NLP\Edliyye\Legal Acts Question Answering\NLP project Notarius\sorter\search\other\documents\embeddings_data", index=False)

In [12]:
embeddings_data = pd.read_csv("E:\Software\Data Science and AI\NLP\Edliyye\Legal Acts Question Answering\NLP project Notarius\sorter\search\other\documents\embeddings_data")

In [13]:
dims = embeddings_data.shape[1]
cells = 1

index_flat = faiss.IndexFlatL2(dims)
clustering_index = faiss.index_cpu_to_all_gpus(index_flat)
idx_l1 = faiss.IndexIVFFlat(clustering_index, dims, cells)

idx_l1.train(np.ascontiguousarray(embeddings_data).astype('float32'))
idx_l1.add(np.ascontiguousarray(embeddings_data).astype('float32'))
idx_l1 = faiss.index_gpu_to_cpu(idx_l1)

In [14]:
faiss.write_index(idx_l1, f"E:\Software\Data Science and AI\NLP\Edliyye\Legal Acts Question Answering\NLP project Notarius\sorter\search\other\models\model.faiss")